In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
import pyspark
from pyspark.sql import SQLContext
sc = pyspark.SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [3]:
#First: filter the restaurant
biz_df = sqlContext.read.json("original_data/business.json")

In [4]:
biz_df.createOrReplaceTempView('biz_table')

In [5]:
biz_res = sqlContext.sql('SELECT * FROM biz_table WHERE categories LIKE \'%Restaurants%\'')

In [6]:
biz_res.createOrReplaceTempView('biz_res_table')

In [7]:
def filter_res(parent,filename,key):
    df = sqlContext.read.json("original_data/"+filename)
    table_name=filename[:-5]+'_table'
    df.createOrReplaceTempView(filename[:-5]+'_table')
    sqlquery = 'SELECT c.* FROM {} c LEFT JOIN {} b ON c.{} = b.{} WHERE b.{} IS NOT NULL'.format(table_name,parent,key,key,key)
    print(sqlquery)
    df_res=sqlContext.sql(sqlquery)
    return df_res

In [8]:
checkin_res = filter_res('biz_res_table','checkin.json','business_id')
review_res = filter_res('biz_res_table','review.json','business_id')
review_res.createOrReplaceTempView('review_res_table')
tip_res = filter_res('review_res_table','tip.json','user_id') #since tip is on individual level, match them with users
user_res = filter_res('review_res_table','user.json','user_id') 

SELECT c.* FROM checkin_table c LEFT JOIN biz_res_table b ON c.business_id = b.business_id WHERE b.business_id IS NOT NULL
SELECT c.* FROM review_table c LEFT JOIN biz_res_table b ON c.business_id = b.business_id WHERE b.business_id IS NOT NULL
SELECT c.* FROM tip_table c LEFT JOIN review_res_table b ON c.user_id = b.user_id WHERE b.user_id IS NOT NULL
SELECT c.* FROM user_table c LEFT JOIN review_res_table b ON c.user_id = b.user_id WHERE b.user_id IS NOT NULL


In [9]:
#sample  business into dev_set and test_set
import pyspark.sql.functions as f
review_res = review_res.withColumn('index_1', f.monotonically_increasing_id())
review_res.createOrReplaceTempView('review_res_table')
sqlquery = 'SELECT * FROM review_res_table ORDER BY RAND(42) LIMIT {}'.format(1000)
review_sample = sqlContext.sql(sqlquery)
review_sample.createOrReplaceTempView('review_sample_table')

sqlquery = 'SELECT a.* FROM review_res_table a LEFT JOIN review_sample_table b ON a.index_1 = b.index_1 WHERE b.index_1 IS NULL '
review_train = sqlContext.sql(sqlquery)
# 


In [10]:
#sample 500 into dev_set and 500 into test_set
from pyspark.sql.functions import desc
review_sample = review_sample.withColumn('index_2', f.monotonically_increasing_id())
review_dev = review_sample.limit(500)
review_test = review_sample.sort(desc("index_2")).limit(500)

In [11]:
review_dev.toPandas().to_csv('filtered_data/review_dev.csv',header = 'true')
review_test.toPandas().to_csv('filtered_data/review_test.csv',header = 'true')

In [12]:
review_dev.show()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+------------+-------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|     index_1|index_2|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+------------+-------+
|wArcCMVnrl_tc9MUL...|   0|2014-05-02 14:22:05|    0|ZtAnERu9OwjIO4ug0...|  3.0|We loved the atmo...|     0|g60bxO0B2I6os1iCX...|223338344704|      0|
|YlwJYpBWa-oL0byTV...|   0|2018-11-03 19:57:36|    0|8b_S0_FDF0zx3sELm...|  1.0|SO SLOW! Mostly E...|     0|MWKALAdfSvzFE_LEj...|335007509918|      1|
|8mIrX_LrOnAqWsB5J...|   0|2016-10-23 19:18:39|    0|IF9A7iGnOLhF3aSlx...|  4.0|So much fun for a...|     0|93NLI_oDOFpDRmgxx...| 17179951328|      2|
|M-bM5y8qdE-Z3mfZe...|   1|2011-03-29 05:07:26|    0|bebMts1G_tBse6zra...|  4.0|I like this pl

In [13]:
#select the record from business checkin tip and users based on business_id(first two), user_id(last two)
setname=['_train','_dev','_test']
def filter_sample(parent,child,key): #parent:the reference table, child: the table needs filtering
    parent.createOrReplaceTempView('p')
    child.createOrReplaceTempView('c') #child xx_res
    sqlquery = 'SELECT c.* FROM c LEFT JOIN p ON c.{} = p.{} WHERE p.{} IS NOT NULL'.format(key,key,key)
    df=sqlContext.sql(sqlquery)
    return df
for i in range(3):
    globals()['biz'+setname[i]] = filter_sample(globals()['review'+setname[i]],biz_res,'business_id')
    globals()['checkin'+setname[i]] = filter_sample(globals()['review'+setname[i]],checkin_res,'business_id')
    globals()['tip'+setname[i]] = filter_sample(globals()['review'+setname[i]],tip_res,'user_id')
    globals()['user'+setname[i]] = filter_sample(globals()['review'+setname[i]],user_res,'user_id')

In [14]:
biz_test.show()

+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|           city|               hours|is_open|     latitude|      longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|322 Adelaide Stre...|[,, u'beer_and_wi...|c35qzLN6ItDpVIoj2...|Soup, Restaurants...|        Toronto|[11:0-22:0, 11:0-...|      1|   43.6475853|    -79.3920316|          Ravi Soups|    M5V 1R1|         332|  4.5|   ON|
|     2004 Crocker Rd|[,,, {'touristy':...|nYvBZYg9rfqWFTYux...|Mexican, Bars, Re...|       Westlake|[11:0-23:0, 11:0-...|  